### import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
#sns.set_style('whitegrid')
import h5py
from PIL import Image
import os
from tqdm.notebook import tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')
sys.path.append('/scratch/xc1490/projects/tmp/python_packages')
sys.path.append('/scratch/xc1490/projects/tmp/python_packages/') #pip install --target=/home/xc1490/home/projects/tmp/python_packages package_name

python main.py  --model iTransformer   --data_path processed_data --hist_time 2 \
--pred_time 1 --batch_size 16 --feature_names XYZ_FEATURE_NAMES \
--n_heads 3 --head_dim 3 --dim_val 9  --n_decoder_layers 2 --n_encoder_layers 2 --pe_mode standard

python main.py  --model iTransformer   --data_path processed_data --hist_time 2 \
--pred_time 0.1 --batch_size 16 --feature_names XYZ_FEATURE_NAMES \
--n_heads 3 --head_dim 3 --d_model 9  --n_decoder_layers 2 --n_encoder_layers 2 --pe_mode standard

python main.py  --model PatchTST   --data_path processed_data --hist_time 2 \
--pred_time 0.1 --batch_size 16 --feature_names XYZ_FEATURE_NAMES \
--n_heads 3 --head_dim 3 --d_model 9  --n_decoder_layers 2 --n_encoder_layers 2 --pe_mode standard

python main.py  --model MyTransformer   --data_path processed_data --hist_time 2 \
--pred_time 0.1 --batch_size 16 --feature_names XYZ_FEATURE_NAMES \
--n_heads 3 --head_dim 3 --d_model 9  --n_decoder_layers 2 --n_encoder_layers 2 --pe_mode standard

In [24]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
#sns.set_style('whitegrid')
import h5py
from PIL import Image
import os
from tqdm.notebook import tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [2]:
cd /scratch/xc1490/projects/FoV/

/scratch/xc1490/projects/FoV


In [14]:
def fov_prediction(model,hist_time, pred_time,batch_size , feature_names,n_heads,head_dim,\
                  d_model,n_decoder_layers,n_encoder_layers,pe_mode,out_suffix,num_epochs,printcommand=True):
    text = '#!/bin/bash'
    text += '\n'
    text += '\n'
    text += '#SBATCH --job-name=fov{}'.format(model)
    text += '\n'
    text += '#SBATCH --nodes=1'
    text += '\n'
    text += '#SBATCH --cpus-per-task=1'
    text += '\n'
    text += '#SBATCH --mem=24GB'
    text += '\n'
    text += '#SBATCH --time=8:00:00'
    text += '\n'
    text += '#SBATCH --gres=gpu:1'
    text += '\n'
    text += 'overlay_ext3=/home/xc1490/home/apps/ddsp.ext3'
    text +='\n'
    text +='singularity exec --nv \\'
    text +='\n'
    text +='    --overlay ${overlay_ext3}:ro \\'
    text +='\n'
    text +='    /scratch/work/public/singularity/cuda11.0-cudnn8-devel-ubuntu18.04.sif \\'
    text +='\n'
    text +='    /bin/bash -c "'
    text +='\n'
    text +='source /ext3/env.sh'
    text +='\n'
    text +='cd /scratch/xc1490/projects/FoV/'
    text +='\n'
    text1 ='python main.py  --model {} --data_path processed_data --hist_time {} \
                        --pred_time {} --batch_size {} --feature_names {} \
                        --n_heads {} --head_dim {} --d_model {}  --n_decoder_layers {} --n_encoder_layers {} --pe_mode {} \
                        --out_suffix {} --num_epochs {}"'.format(\
                      model,hist_time, pred_time,batch_size , feature_names,n_heads,head_dim,\
                  d_model,n_decoder_layers,n_encoder_layers,pe_mode,out_suffix,num_epochs)
    text += text1
    text +='\n'
    
    job_file = '/scratch/xc1490/projects/FoV/jobs/model_{}_hist_{}_pred_{}_bs_{}_feat_{}_n_heads_{}_hedim_{}_dmod_{}_declay_{}_enc_lay_{}_pe_{}_{}_epoch_{}'.format(
                        model,hist_time, pred_time,batch_size , feature_names.split('_')[0],n_heads,head_dim,\
                  d_model,n_decoder_layers,n_encoder_layers,pe_mode,out_suffix,num_epochs)

    f= open(job_file,"w+")
    f.write(text)
    f.close()
    if printcommand:
        print ('sbatch ',job_file)
    return text1

In [10]:
model='MyTransformer'
hist_time, pred_time = 2,0.1
batch_size = 8
feature_names = 'XYZ_FEATURE_NAMES'
n_heads = 3
head_dim = 3
d_model = 9
n_decoder_layers = 2
n_encoder_layers = 2
pe_mode = 'standard'
out_suffix = '20240313'
num_epochs = 2000
for model in ['MyTransformer','TimesNet','iTransformer','PatchTST','Reformer']:
    for times in [[2,0.1],[1,0.1],[0.5,0.1],[2,0.5],[1,0.5],[0.5,0.5],[0.1,0.1],[2,2],[1,1]]:
        hist_time, pred_time = times
        text  = fov_prediction(model,hist_time, pred_time,batch_size , feature_names,n_heads,head_dim,\
                      d_model,n_decoder_layers,n_encoder_layers,pe_mode,out_suffix,num_epochs)

sbatch  /scratch/xc1490/projects/FoV/jobs/model_MyTransformer_hist_2_pred_0.1_bs_8_feat_XYZ_n_heads_3_hedim_3_dmod_9_declay_2_enc_lay_2_pe_standard_20240313_epoch_2000
sbatch  /scratch/xc1490/projects/FoV/jobs/model_MyTransformer_hist_1_pred_0.1_bs_8_feat_XYZ_n_heads_3_hedim_3_dmod_9_declay_2_enc_lay_2_pe_standard_20240313_epoch_2000
sbatch  /scratch/xc1490/projects/FoV/jobs/model_MyTransformer_hist_0.5_pred_0.1_bs_8_feat_XYZ_n_heads_3_hedim_3_dmod_9_declay_2_enc_lay_2_pe_standard_20240313_epoch_2000
sbatch  /scratch/xc1490/projects/FoV/jobs/model_MyTransformer_hist_2_pred_0.5_bs_8_feat_XYZ_n_heads_3_hedim_3_dmod_9_declay_2_enc_lay_2_pe_standard_20240313_epoch_2000
sbatch  /scratch/xc1490/projects/FoV/jobs/model_MyTransformer_hist_1_pred_0.5_bs_8_feat_XYZ_n_heads_3_hedim_3_dmod_9_declay_2_enc_lay_2_pe_standard_20240313_epoch_2000
sbatch  /scratch/xc1490/projects/FoV/jobs/model_MyTransformer_hist_0.5_pred_0.5_bs_8_feat_XYZ_n_heads_3_hedim_3_dmod_9_declay_2_enc_lay_2_pe_standard_20240313

In [11]:
text

'python main.py  --model Reformer --data_path processed_data --hist_time 1                         --pred_time 1 --batch_size 8 --feature_names XYZ_FEATURE_NAMES                         --n_heads 3 --head_dim 3 --d_model 9  --n_decoder_layers 2 --n_encoder_layers 2 --pe_mode standard                         --out_suffix 20240313 --num_epochs 2000"'

In [37]:
def fov_prediction_multiple(python_commands,suffix=''):
    text = '#!/bin/bash'
    text += '\n'
    text += '\n'
    text += '#SBATCH --job-name=fov{}'.format(model)
    text += '\n'
    text += '#SBATCH --nodes=1'
    text += '\n'
    text += '#SBATCH --cpus-per-task=1'
    text += '\n'
    text += '#SBATCH --mem=24GB'
    text += '\n'
    text += '#SBATCH --time=16:00:00'
    text += '\n'
    text += '#SBATCH --gres=gpu:1'
    text += '\n'
    text += 'overlay_ext3=/scratch/xc1490/apps/ddsp.ext3'
    text +='\n'
    text +='singularity exec --nv \\'
    text +='\n'
    text +='    --overlay ${overlay_ext3}:ro \\'
    text +='\n'
    text +='    /scratch/work/public/singularity/cuda11.0-cudnn8-devel-ubuntu18.04.sif \\'
    text +='\n'
    text +='    /bin/bash -c "'
    text +='\n'
    text +='source /ext3/env.sh'
    text +='\n'
    text +='cd /scratch/xc1490/projects/FoV/'
    text +='\n'
    for python_command in python_commands:
        text +=  python_command + ' &' + '\n'
    text +='\n'
    text += 'wait'
    job_file = '/scratch/xc1490/projects/FoV/jobs/{}_{}.sbatch'.format(datetime.now().strftime("%m-%d-%Y-%H"),suffix)

    f= open(job_file,"w+")
    f.write(text)
    f.close()
    print ('sbatch ',job_file)

In [38]:
model='MyTransformer'
hist_time, pred_time = 2,0.1
batch_size = 16
feature_names = 'XYZ_FEATURE_NAMES'
n_heads = 3
head_dim = 3
d_model = 9
n_decoder_layers = 2
n_encoder_layers = 2
pe_mode = 'standard'
out_suffix = '20240313'
num_epochs = 2000
texts = []
for model in ['MyTransformer','TimesNet','iTransformer','PatchTST','Reformer']:
    for times in [[2,0.1],[1,0.1],[0.5,0.1],[2,0.5],[1,0.5],[0.5,0.5],[0.1,0.1],[2,2],[1,1]]:
        hist_time, pred_time = times
        text  = fov_prediction(model,hist_time, pred_time,batch_size , feature_names,n_heads,head_dim,\
                      d_model,n_decoder_layers,n_encoder_layers,pe_mode,out_suffix,num_epochs,printcommand=False)
        texts.append(text[:-1])

In [39]:
for i in range(len(texts)//4):
    #print (i, texts[i*4:min(i*4+4,len(texts))])
    fov_prediction_multiple(texts[i*4:min(i*4+4,len(texts))],suffix=str(i))

sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_0.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_1.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_2.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_3.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_4.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_5.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_6.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_7.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_8.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_9.sbatch
sbatch  /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_10.sbatch


In [40]:
cat /scratch/xc1490/projects/FoV/jobs/03-13-2024-20_0.sbatch

#!/bin/bash

#SBATCH --job-name=fovReformer
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=24GB
#SBATCH --time=12:00:00
#SBATCH --gres=gpu:1
overlay_ext3=/scratch/xc1490/apps/ddsp.ext3
singularity exec --nv \
    --overlay ${overlay_ext3}:ro \
    /scratch/work/public/singularity/cuda11.0-cudnn8-devel-ubuntu18.04.sif \
    /bin/bash -c "
source /ext3/env.sh
cd /scratch/xc1490/projects/FoV/
python main.py  --model MyTransformer --data_path processed_data --hist_time 2                         --pred_time 0.1 --batch_size 16 --feature_names XYZ_FEATURE_NAMES                         --n_heads 3 --head_dim 3 --d_model 9  --n_decoder_layers 2 --n_encoder_layers 2 --pe_mode standard                         --out_suffix 20240313 --num_epochs 2000 &
python main.py  --model MyTransformer --data_path processed_data --hist_time 1                         --pred_time 0.1 --batch_size 16 --feature_names XYZ_FEATURE_NAMES                         --n_heads 3 --head_dim 3 --d_model 9  --n_dec